on its own

In [1]:
from haystack.components.builders import AnswerBuilder

from haystack.Concepts.getting_start import prompt_template, document_store

builder = AnswerBuilder(pattern="Answer: (.*)")
builder.run(query="What's the answer?", replies=["This is an argument. Answer: This is the answer."])

{'answers': [GeneratedAnswer(data='This is the answer.', query="What's the answer?", documents=[], meta={'all_messages': ['This is an argument. Answer: This is the answer.']})]}

in a pipeline

In [10]:
from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever
from haystack.components.generators.chat import HuggingFaceAPIChatGenerator
from haystack.utils.hf import HFGenerationAPIType
from haystack.components.builders.answer_builder import AnswerBuilder
from haystack.components.builders.chat_prompt_builder import ChatPromptBuilder
from haystack.dataclasses import ChatMessage
from haystack.utils import Secret
from haystack.dataclasses import Document

In [11]:
prompt_template = [
    ChatMessage.from_system("You are a helpful assistant."),
    ChatMessage.from_user(
        "Given these documents, answer the question.\nDocuments:\n"
        "{% for doc in documents %}{{ doc.content }}{% endfor %}\n"
        "Question: {{query}}\nAnswer:"
    ),
]



In [12]:
docs = [Document(content="The capital of France is Paris"), Document(content="The capital of England is London")]
document_store = InMemoryDocumentStore()
document_store.write_documents(docs)

p = Pipeline()
p.add_component(instance=InMemoryBM25Retriever(document_store=document_store), name="retriever")
p.add_component(instance=ChatPromptBuilder(template=prompt_template, required_variables={"query", "documents"}), name="prompt_builder")
p.add_component(instance=HuggingFaceAPIChatGenerator(api_type=HFGenerationAPIType.SERVERLESS_INFERENCE_API,
                                  api_params={"model": "Qwen/Qwen2.5-7B-Instruct",
                                             "provider": "together"},
                                  token=Secret.from_token("")), name="llm")
p.add_component(instance=AnswerBuilder(), name="answer_builder")
p.connect("retriever", "prompt_builder.documents")
p.connect("prompt_builder", "llm.messages")
p.connect("llm.replies", "answer_builder.replies")
p.connect("retriever", "answer_builder.documents")

🚅 Components
  - retriever: InMemoryBM25Retriever
  - prompt_builder: ChatPromptBuilder
  - llm: HuggingFaceAPIChatGenerator
  - answer_builder: AnswerBuilder
🛤️ Connections
  - retriever.documents -> prompt_builder.documents (list[Document])
  - retriever.documents -> answer_builder.documents (list[Document])
  - prompt_builder.prompt -> llm.messages (list[ChatMessage])
  - llm.replies -> answer_builder.replies (list[ChatMessage])

In [13]:
query = "What is the capital of France?"
result = p.run(
    {
        "retriever": {"query": query},
        "prompt_builder": {"query": query},
        "answer_builder": {"query": query},
    }
)
result

{'answer_builder': {'answers': [GeneratedAnswer(data='The capital of France is Paris.', query='What is the capital of France?', documents=[Document(id=23d04d82c60ead672e09b57072286ffdfa3af800c86c9aab7ae7e70560e52434, content: 'The capital of France is Paris', meta: {'source_index': 1}, score: 1.600237583702734), Document(id=17ba2169275b86fdbfd26cccec33c76b35e1303de86f39797cfe8f16c2865f58, content: 'The capital of England is London', meta: {'source_index': 2}, score: 1.2536639934227616)], meta={'model': 'Qwen/Qwen2.5-7B-Instruct', 'finish_reason': 'tool_calls', 'index': 0, 'usage': {'prompt_tokens': 53, 'completion_tokens': 8}, 'all_messages': [ChatMessage(_role=<ChatRole.ASSISTANT: 'assistant'>, _content=[TextContent(text='The capital of France is Paris.')], _name=None, _meta={'model': 'Qwen/Qwen2.5-7B-Instruct', 'finish_reason': 'tool_calls', 'index': 0, 'usage': {'prompt_tokens': 53, 'completion_tokens': 8}})]})]}}

In [1]:
from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever
from haystack.components.generators.chat import HuggingFaceAPIChatGenerator
from haystack.utils.hf import HFGenerationAPIType
from haystack.components.builders.chat_prompt_builder import ChatPromptBuilder
from haystack.components.builders.answer_builder import AnswerBuilder
from haystack.utils import Secret
from haystack.dataclasses import ChatMessage
from haystack.dataclasses import Document

prompt_template = [
    ChatMessage.from_system("You are a helpful assistant."),
    ChatMessage.from_user(
        "Given these documents, answer the question.\nDocuments:\n"
        "{% for doc in documents %}{{ doc.content }}{% endfor %}\n"
        "Question: {{query}}\nAnswer:"
    )
]

docs = [Document(content="The capital of France is Paris"), Document(content="The capital of England is London")]
document_store = InMemoryDocumentStore()
document_store.write_documents(docs)

p = Pipeline()
p.add_component(instance=InMemoryBM25Retriever(document_store=document_store), name="retriever")
p.add_component(instance=ChatPromptBuilder(template=prompt_template, required_variables={"query", "documents"}), name="prompt_builder")
p.add_component(instance=HuggingFaceAPIChatGenerator(api_type=HFGenerationAPIType.SERVERLESS_INFERENCE_API,
                                  api_params={"model": "Qwen/Qwen2.5-7B-Instruct",
                                             "provider": "together"},
                                  token=Secret.from_token("")), name="llm")
p.add_component(instance=AnswerBuilder(), name="answer_builder")
p.connect("retriever", "prompt_builder.documents")
p.connect("prompt_builder", "llm.messages")
p.connect("llm.replies", "answer_builder.replies")
p.connect("retriever", "answer_builder.documents")

query = "What is the capital of France?"
result = p.run(
    {
        "retriever": {"query": query},
        "prompt_builder": {"query": query},
        "answer_builder": {"query": query},
    }
)

print(result)

/home/eyad/haystack/hastack_venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'answer_builder': {'answers': [GeneratedAnswer(data='The capital of France is Paris.', query='What is the capital of France?', documents=[Document(id=23d04d82c60ead672e09b57072286ffdfa3af800c86c9aab7ae7e70560e52434, content: 'The capital of France is Paris', meta: {'source_index': 1}, score: 1.600237583702734), Document(id=17ba2169275b86fdbfd26cccec33c76b35e1303de86f39797cfe8f16c2865f58, content: 'The capital of England is London', meta: {'source_index': 2}, score: 1.2536639934227616)], meta={'model': 'Qwen/Qwen2.5-7B-Instruct', 'finish_reason': 'tool_calls', 'index': 0, 'usage': {'prompt_tokens': 53, 'completion_tokens': 8}, 'all_messages': [ChatMessage(_role=<ChatRole.ASSISTANT: 'assistant'>, _content=[TextContent(text='The capital of France is Paris.')], _name=None, _meta={'model': 'Qwen/Qwen2.5-7B-Instruct', 'finish_reason': 'tool_calls', 'index': 0, 'usage': {'prompt_tokens': 53, 'completion_tokens': 8}})]})]}}
